# Large Language Models

A number of Large Language Models (LLMs) are available in `langchain_dartmouth`. 

LLMs in this library generally come in three flavors:

- **Baseline completion models:** 
  - These models are trained to simply continue the given prompt by adding the next token.
- **Instruction-tuned chat models:** 
  - These models are built on baseline completion models, but further trained using a specific prompt format to allow a conversational back-and-forth.
- **Commercial third-party chat models:**
  - Dartmouth offers limited access to various third-party commercial models, e.g., OpenAI's GPT-4o or Anthropic's Claude. Daily token limits per user apply.

Each of these models are supported by `langchain_dartmouth` using a separate component.

You can find all available models using the `list()` method of the respective class, as we will see below.

Let's explore these components! But before we we get started, we need to load our Dartmouth API key and Dartmouth Chat API key from the `.env` file:

In [ ]:
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

## Baseline Completion Models

Baseline completion models are trained to simply continue the given prompt by adding the next token. The continued prompt is then considered the next input to the model, which extends it by another token. This continues until a specified maximum number of tokens have been added, or until a special token called a _stop token_ is generated.

A popular use-case for completion models is to generate code. Let's try an example and have the LLM generate a function based on its signature!

All baseline completion models are available through the component `DartmouthLLM` in the submodule `langchain_dartmouth.llms`, so we first need to import that class:

In [5]:
from langchain_dartmouth.llms import DartmouthLLM

We can find out, which models are available, by using the static method `list()`:

```{note}
   A static method is a function that is defined on the class itself, not on an instance of the class. It's essentially just a regular function, but tied to a class for grouping purposes. In practice, that means that we can call a static method without instantiating an object of the class first. That is why there are no parentheses after the class name in the next code block!
```

In [ ]:
DartmouthLLM.list()

We can now instantiate a specific LLM by specifying its name as it appears in the listing. Since the model will generate the continuation of our prompt, it usually makes sense to repeat our prompt in the response, which we can request by setting the parameter `return_full_text` to `True`:

In [ ]:
llm = DartmouthLLM(model_name="codellama-13b-python-hf", return_full_text=True)

We can now send a prompt to the model and receive its response by using the `invoke()` method:

In [ ]:
response = llm.invoke("def remove_digits(s: str) -> str:")
print(response)

Since they are only trained to continue the given prompt, completion models are not great at responding to chat-like prompts:

In [ ]:
response = llm.invoke("How can I define a class in Python?")
print(response)

As we can see, the model just continues the prompt in a way that is similar to what it has seen during its training. If we want to use it in a conversational way, we need to use an instruction-tuned chat model.

## Instruction-Tuned Chat Models

Instruction-tuned chat models are trained to follow a specific set of instructions that the model is expected to follow. These models can be used in conversational scenarios, where the user asks the model questions and the model replies with answers. The model will not just continue the prompt but also understand the context of the conversation preceding the prompt. To achieve this, baseline completion models are fine-tuned (i.e., further trained) on conversational text material that is formatted following a particular template. That is why we often see multiple variants of an LLM: the base model and the instruct version (see, e.g., [CodeLlama](https://huggingface.co/meta-llama/CodeLlama-13b-Instruct-hf)).

Let's see what happens if we ask an instruction-tuned model our question from the previous section:

In [ ]:
llm = DartmouthLLM(model_name="codellama-13b-instruct-hf")
response = llm.invoke("How can I define a class in Python?")
print(response)

Well, that does not seem very helpful... What went wrong here?

The problem is that the prompt we use during inference (when we invoke the model) needs to follow the same format that was used during the instruction-tuning. This format is not the same for every model! Let's try our prompt again using [CodeLlama's Instructions format](https://llama.meta.com/docs/model-cards-and-prompt-formats/meta-code-llama/):

In [ ]:
response = llm.invoke("<s>[INST] How can I define a class in Python? [/INST] ")

print(response)

That looks a lot better! 

```{note}

You may notice that the last sentence gets cut off. This is due to the default value for the maximum number of generated tokens, which may be too low. You can set a higher limit when you instantiate the `DartmouthLLM` object. Check [the API reference](https://dartmouth.github.io/langchain-dartmouth/api.html#langchain_dartmouth.llms.DartmouthLLM) for more information.
```


Managing the prompt format can quickly get tedious, especially if you want to switch between different models. Fortunately, the `ChatDartmouth` component handles the prompt formatting "under-the-hood" and we can just pass the actual message when we invoke it:

In [ ]:
from langchain_dartmouth.llms import ChatDartmouth

llm = ChatDartmouth(model_name="llama-3-1-8b-instruct")
response = llm.invoke("How can I define a class in Python?")

print(response.content)

That looks a lot better! 

```{note}
`ChatDartmouth` returns more than just a raw string: It returns an `AIMessage` object, which you can learn more about in [LangChain's API reference](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.ai.AIMessage.html).

We will see more of these message objects in the recipe on [prompts](06-prompt-basics.ipynb)!
```




By the way, just like with `DartmouthLLM`, we can get a list of the available chat models using the static method `list()`:

In [ ]:
ChatDartmouth.list()

## Third-party chat models

In addition to the locally-deployed, open-source models, Dartmouth also offers access to various third-party chat models. These models are available through the `ChatDartmouthCloud` class. 

```{note}
   Remember: You need a separate API key for `ChatDartmouthCloud`. Follow [the instructions](https://rcweb.dartmouth.edu/~d20964h/2024-12-11-dartmouth-chat-api/api_key/) to get yours, and then store it in an environment variable called `DARTMOUTH_CHAT_API_KEY`. 
```

In [15]:
from langchain_dartmouth.llms import ChatDartmouthCloud

As with the other classes, we can list the available models using the static method `list()`:

In [ ]:
ChatDartmouthCloud.list()

Using the class works just like with the other two varieties:

In [ ]:
llm = ChatDartmouthCloud(model_name="openai.gpt-4o-mini-2024-07-18")

response = llm.invoke("Who are you?")

print(response.content)

```{warning}
   The models available through `ChatDartmouthCloud` are commercial, third-party models. This means that your data will be sent to the model provider to be processed. If you have privacy concerns, please reach out to [Research Computing](mailto:research.computing@dartmouth.edu?subject=API%20terms%20of%20use%20for%20Dartmouth%20Chat%20model) to obtain a copy of the terms of use for the model you are interested in.
```

```{note}
   Dartmouth pays for a significant daily token allotment per user, but eventually you may hit a limit. If you need a larger volume of tokens for your project, please [reach out](mailto:research.computing@dartmouth.edu?subject=Dartmouth%20Chat%20token%20limit)!
```

## Summary

In this recipe, we have learned how to use the `DartmouthLLM`, `ChatDartmouth`, `ChatDartmouthCloud` components. Which one to use depends on whether you are working with a baseline completion model, a local instruction-tuned chat model, or a Cloud-hosted third-party chat model:

Baseline completion models can only be used with `DartmouthLLM`. Local instruction-tuned chat models should be used with `ChatDartmouth` so the correct prompt format is applied automatically. For commercial third-party chat models, use `ChatDartmouthCloud`.

You can also use `DartmouthLLM` with an instruction-tuned model, if you want full control over the exact string that is sent to the model. In that case, however, you might see unexpected responses if the prompt format is not correct.